## Prepare Flight Price Data

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

Load training configuration with Omegaconf

In [2]:
from alibaba_ai_task.tools.omni_tools import get_support_data_dir
import os.path as osp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from loguru import logger
import sys

logger.remove()
log_format = "{message}"
ds_logger_id = logger.add(sys.stderr, format=f"<level>{log_format}</level>", enqueue=True)

In [3]:
from alibaba_ai_task.tools.omni_tools import get_support_data_dir
from alibaba_ai_task.train.trainer import APOTrainer

support_dir = get_support_data_dir()

logger.info(support_dir)

/is/ps3/nghorbani/code-repos/alibab_ai_task/support_data


In [4]:
       
cfg = APOTrainer.prepare_train_cfg(
    **{
    'apo.expr_id': 'V01',

    'apo.data_id': 'V01',
        
    'dirs.support_base_dir': support_dir,
    'dirs.work_base_dir': '/home/nghorbani/Desktop/alibaba_ai_task',

    'data_parms.history_length': 7,
    'data_parms.future_length': 7
})


print(cfg)

logger.info(cfg.dirs.dataset_dir)

{'apo': {'expr_id': 'V01', 'data_id': 'V01'}, 'dirs': {'work_base_dir': '/home/nghorbani/Desktop/alibaba_ai_task', 'support_base_dir': '/is/ps3/nghorbani/code-repos/alibab_ai_task/support_data', 'work_dir': '${dirs.work_base_dir}/training_experiments/${apo.expr_id}/${apo.data_id}', 'log_dir': '${dirs.work_dir}/logs', 'log_fname': '${dirs.log_dir}/${apo.expr_id}_${apo.data_id}.log', 'cfg_fname': '${dirs.work_dir}/${apo.expr_id}_${apo.data_id}.yaml', 'dataset_dir': '${dirs.work_base_dir}/data/${apo.expr_id}'}, 'train_parms': {'batch_size': '???', 'num_workers': 5, 'loss_weights': {'price': 1.0}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 5e-05, 'betas': [0.9, 0.999]}}, 'lr_scheduler': {'type': 'ReduceLROnPlateau', 'args': {'verbose': True, 'patience': 3}}, 'early_stopping': {'monitor': 'val_loss', 'min_delta': 0.0, 'patience': 8, 'verbose': True, 'mode': 'min'}}, 'data_parms': {'history_length': 7, 'future_length': 7}, 'model_parms': {'labeler': {'enable_transfor

/home/nghorbani/Desktop/alibaba_ai_task/data/V01


In [5]:
init_data_fname = osp.join(support_dir, 'mhd_thr.csv')
logger.info(f'Loading initial data from {init_data_fname}')
df = pd.read_csv(init_data_fname, names=['row','log_data','date', 'route', 'price', 'al'], 
                 skiprows=[0], usecols=[0,2,3,4,5], index_col='row')
df['date'] = pd.to_datetime(df['date'])# - pd.to_timedelta(7, unit='d')
df

Loading initial data from /is/ps3/nghorbani/code-repos/alibab_ai_task/support_data/mhd_thr.csv


,date,route,price,al
row,,,,
4,2018-08-21,MHDTHR,3663000.0,1
5,2018-08-22,MHDTHR,3663000.0,1
6,2018-08-23,MHDTHR,3663000.0,1
7,2018-08-23,THRMHD,3663000.0,1
8,2018-08-13,THRMHD,3378000.0,1
...,...,...,...,...
148376,2020-03-12,MHDTHR,4532000.0,1
148377,2020-03-12,THRMHD,4532000.0,1
148378,2020-03-13,MHDTHR,4532000.0,1


Set max number of rows to be shown to the number of unique airlines

In [6]:
num_airlines = len(df.groupby('al')['al'].agg(['count']))
logger.info(f'Number of unique AL: {num_airlines}')
pd.set_option("max_rows", num_airlines)

Number of unique AL: 15


In [7]:
df.groupby(['al'])['al'].agg(['count'])

,count
al,
1,10197
2,6967
3,5961
4,4707
5,3337
6,2110
7,2007
8,2389
9,2694


### Remove duplicate prices for the same date/route/al.
Note that there could actually be multiple flights for an airline in a day.

This can happen when flights are on different hours. For the task at hand we would take the median price since we handle data at daily basis. Notice the drop in row count after the following cell.

In [8]:
df = df.groupby(['date','al', 'route'])['price'].agg('median').reset_index()
df

,date,al,route,price
0,2018-08-13,1,THRMHD,3378000.0
1,2018-08-14,1,MHDTHR,3102000.0
2,2018-08-14,1,THRMHD,3176000.0
3,2018-08-16,1,MHDTHR,2826000.0
4,2018-08-16,1,THRMHD,3378000.0
...,...,...,...,...
5436,2020-03-14,1,THRMHD,4613000.0
5437,2020-03-15,1,MHDTHR,4080000.0
5438,2020-03-15,1,THRMHD,4613000.0
5439,2020-03-16,1,MHDTHR,4080000.0


## Filter noisy data


### Remove airlines with very few records.

In [9]:
df = df.groupby('al').filter(lambda x : len(x)>10)
df.groupby(['al'])['al'].agg(['count'])

,count
al,
1,1055
2,415
3,393
4,408
5,423
6,413
7,423
8,410
9,407


### Skip the negative price records

In [10]:
agg_func_math = {
    'price':
    ['mean', 'median', 'min', 'max', 'std', 'var', ],
}
df.groupby(['al']).agg(agg_func_math).round(2)


price                                                              
          mean      median         min         max         std           var
al                                                                          
1   3981563.56  3980000.00       -1.00  6631500.00   965086.66  9.313923e+11
2   4198515.34  4230500.00  1737000.00  6407000.00   857391.27  7.351198e+11
3   4163444.02  4119000.00  2213333.33  7100666.67   753841.81  5.682775e+11
4   4171884.50  4121375.00  2310000.00  6815500.00   732534.40  5.366066e+11
5   4162069.22  4088200.00  2398000.00  6619800.00   722900.77  5.225855e+11
6   4272594.89  4204833.33  2168666.67  6870333.33   843085.60  7.107933e+11
7   4301174.90  4294428.57  1734571.43  6543857.14   993697.24  9.874342e+11
8   4155161.13  4175500.00  1515812.50  6403250.00  1077285.00  1.160543e+12
9   3986647.33  3984222.22  1631111.11  6555500.00  1095038.84  1.199110e+12
10  3754825.77  3708100.00  1601600.00  6222900.00  1030004.46  1.060909e+12
11  3679089.84  3671636.36  1507127.27  6144318.18  1019720.09  1.039829e+12
12  3629089.70  3552291.67  1543947.92  5834041.67   973749.14  9.481874e+11
13  3538942.06  3561230.77  1610500.00  5520692.31   888312.92  7.890998e+11
14  3652562.07  3515285.71  2097071.43  5521571.43   952648.66  9.075395e+11

In [11]:
neg_price = df.price<=0
logger.info(f'Number of negative price records: {sum(neg_price)}')
df = df.loc[~neg_price]
logger.info(f'Number of new records: {len(df)}')

Number of negative price records: 1
Number of new records: 5439


In [12]:
routes = list(df.groupby(['route'])['route'].agg(['count']).reset_index()['route'])
logger.info(routes)
df.groupby(['route'])['route'].agg(['count'])

['MHDTHR', 'THRMHD']


,count
route,
MHDTHR,2722
THRMHD,2717


In [13]:
df.groupby(['al', 'route'])['route'].agg('count')

al  route 
1   MHDTHR    526
    THRMHD    528
2   MHDTHR    210
    THRMHD    205
3   MHDTHR    197
             ... 
12  THRMHD    113
13  MHDTHR     59
    THRMHD     36
14  MHDTHR      7
    THRMHD     14
Name: route, Length: 28, dtype: int64

In [14]:
for route in routes:
    logger.info('\n---------------------\n', route)
    logger.info(df[df.route==route].groupby(['al'])['al'].agg('count'))


---------------------

al
1     526
2     210
3     197
4     205
5     210
6     199
7     206
8     200
9     205
10    205
11    167
12    126
13     59
14      7
Name: al, dtype: int64

---------------------

al
1     528
2     205
3     196
4     203
5     213
6     214
7     217
8     210
9     202
10    196
11    170
12    113
13     36
14     14
Name: al, dtype: int64


In [15]:
# import seaborn as sns
# # Use seaborn style defaults and set the default figure size
# sns.set(rc={'figure.figsize':(11, 4)})

In [16]:
# plt.close('all')
# plt.clf()
# plt.cla()
# axes_MHDTHR = df.groupby('al').plot(y='price', legend=False)

In [17]:
# recalculate number of unique airlines
num_airlines = len(df.groupby('al')['al'].agg(['count']))

# Prepare Training Data
Group data in weeks for training

In [18]:
logger.info(df._info_axis)

Index(['date', 'al', 'route', 'price'], dtype='object')


In [19]:
from datetime import datetime, timedelta


def retrieve_week_data(start_date, route):
    end_date = start_date + timedelta(days=cfg.data_parms.history_length+cfg.data_parms.future_length)
    mask = (df["date"] >= start_date) & (df["date"] < end_date) & (df["route"]==route)

    return df.loc[mask].copy()

def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta
        

min_date = df['date'].min()#+timedelta(days=40)
max_date = df['date'].max()-timedelta(days=14)

weekly_grouped_data = []
for start_date in datetime_range(min_date, max_date, timedelta(days=1)):
#     print(start_date)
    for route in routes:
        data = retrieve_week_data(start_date, route)
        #print(len(data))#, len(data)/(14*7.))
        weekly_grouped_data.append(data)
logger.info(f'{len(weekly_grouped_data)} data points')

1134 data points


In [20]:
id_max_d = np.argmax([len(a)/(cfg.data_parms.history_length*float(num_airlines)) for a in weekly_grouped_data])
value_max_d = np.max([len(a)/(cfg.data_parms.history_length*float(num_airlines)) for a in weekly_grouped_data])                    

In [21]:
max_d = weekly_grouped_data[id_max_d]
max_d.groupby(['date','al', 'route'])['al'].agg('count').max()#.reset_index()

1

In [22]:
max_d[(max_d['al'] == 1)]

,date,al,route,price
2934,2019-08-07,1,MHDTHR,3809000.0
2958,2019-08-08,1,MHDTHR,5084000.0
2984,2019-08-09,1,MHDTHR,3591000.0
3009,2019-08-10,1,MHDTHR,3240000.0
3034,2019-08-11,1,MHDTHR,2253000.0
3059,2019-08-12,1,MHDTHR,4470000.0
3083,2019-08-13,1,MHDTHR,3377000.0
3106,2019-08-14,1,MHDTHR,5221000.0
3130,2019-08-15,1,MHDTHR,4120000.0
3155,2019-08-16,1,MHDTHR,3591000.0


In [23]:
weekly_grouped_data[1]

,date,al,route,price
0,2018-08-13,1,THRMHD,3378000.0
2,2018-08-14,1,THRMHD,3176000.0
4,2018-08-16,1,THRMHD,3378000.0
6,2018-08-17,1,THRMHD,3102000.0
8,2018-08-20,1,THRMHD,3520500.0
10,2018-08-21,1,THRMHD,4140500.0
12,2018-08-22,1,THRMHD,4744000.0
14,2018-08-23,1,THRMHD,3663000.0
16,2018-08-24,1,THRMHD,3378000.0
18,2018-08-25,1,THRMHD,3520500.0


## Add Extra Features
**availability**: Whether price for that date/airline is available

## Data Splits

In [24]:
data_size = len(weekly_grouped_data)

data_ids = list(range(data_size))
import random
random.shuffle(data_ids)
test_size = int(0.1*data_size)
vald_size = int(0.1*data_size)
train_size = int(0.8*data_size)
data_split_ids = {'test': data_ids[:test_size],
               'vald': data_ids[test_size:(test_size+vald_size)],
               'train': data_ids[(test_size+vald_size):],
              }
logger.info('Various splits: {} = {} data points'.format({k:len(v) for k,v in data_split_ids.items()}, 
                                                   sum([len(v) for v in data_split_ids.values()])))


Various splits: {'test': 113, 'vald': 113, 'train': 908} = 1134 data points


In [25]:
from collections import OrderedDict
num_feats = 2
data_splits = {}
for split_name, data_ids in data_split_ids.items():
#     if split_name != 'train': continue
#     print(split_name)
    prices_all = []
    for row_id, data_id in enumerate(data_ids):
        prices = np.zeros([cfg.data_parms.history_length+cfg.data_parms.future_length, num_airlines, num_feats])

        dpoint = weekly_grouped_data[data_id]
        if len(dpoint) == 0: continue
        dpoint['dd'] = dpoint['date'] - dpoint['date'].min()

        time_ids = np.array([a.days for a in dpoint['dd']])

        al_ids = np.array([a-1 for a in dpoint['al']])
        prices[time_ids, al_ids, 0] = dpoint['price']

        prices[:,:,1] = (prices[:,:,0]!=0)
        prices_all.append(prices.copy())
    data_splits[split_name] = np.stack(prices_all)
    


In [26]:
logger.info('Various splits: {}'.format({k:v.shape for k,v in data_splits.items()}))
# N x time_length x num_airlines x num_feats

Various splits: {'test': (113, 14, 14, 2), 'vald': (111, 14, 14, 2), 'train': (904, 14, 14, 2)}


## Write Training Data to Disk

In [27]:
import torch
from alibaba_ai_task.tools.omni_tools import makepath
for split_name, data in data_splits.items():
        data_dict = {
#             'price_history': data[:,:cfg.data_parms.history_length],
#             'price_future': data[:,cfg.data_parms.future_length:],
            'price': data,
                    }
        for k, v in data_dict.items():
            outfname = makepath(cfg.dirs.dataset_dir, split_name, f'{k}.pt', isfile=True)
            if osp.exists(outfname): 
                logger.info(f'Already exists!: {outfname}: {v.shape}')
                continue
            v_pt = torch.from_numpy(np.asarray(v))
            torch.save(v_pt, outfname)
            logger.success(f'Created {outfname}: {v.shape}')

Already exists!: /home/nghorbani/Desktop/alibaba_ai_task/data/V01/test/price.pt: (113, 14, 14, 2)
Already exists!: /home/nghorbani/Desktop/alibaba_ai_task/data/V01/vald/price.pt: (111, 14, 14, 2)
Already exists!: /home/nghorbani/Desktop/alibaba_ai_task/data/V01/train/price.pt: (904, 14, 14, 2)


### Todo: Data Normalization
- does it make sense to do input data normalization? Probably it should be per route per airline.
- alternatively you can use a BN at the input.